In [1]:
#Import the libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Import Statsmodels
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
from statsmodels.tools.eval_measures import rmse, aic
from scipy import stats

In [2]:
actuals = pd.read_csv('actuals_2nd.csv')
actuals = actuals.set_index('k1')
actuals.head()

,R0R4_3M,"UK, GDP (q/q)","UK, GDP (y/y)","UK, unemployment rate","UK, inflation (HICP)","UK, home prices (y/y)"
k1,,,,,,
Q3 08,0.001099,0.088300,-1.154803,0.3,0.312441,-2.259206
Q4 08,-0.001501,0.507285,-0.145744,0.0,-2.326910,-0.605646
Q1 09,0.000616,0.819769,0.301229,0.2,0.199405,2.874636
Q2 09,0.000414,1.185762,2.490070,0.0,-0.147302,3.608204
Q3 09,-0.000789,-1.189434,1.274260,-0.6,0.355976,5.502259


In [3]:
model = VAR(actuals)

/Users/pablogtorres/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'


In [5]:
x = model.select_order(maxlags=5)
y = pd.DataFrame(x.summary())
y

,0,1,2,3,4
0,,AIC,BIC,FPE,HQIC
1,0,-19.56,-19.32,3.191e-09,-19.47
2,1,-20.83,-19.11,9.110e-10,-20.20
3,2,-21.25,-18.06,6.611e-10,-20.08
4,3,-28.41,-23.74,6.786e-13,-26.69
5,4,-30.61,-24.47*,1.375e-13*,-28.35
6,5,-31.56*,-23.94,1.886e-13,-28.75*


In [6]:
y = y.rename(columns=y.iloc[0]).drop(y.index[0])
y.drop(y.columns[0], axis=1, inplace=True)
cols = ['AIC', 'BIC', 'FPE', 'HQIC']
y.columns = cols
y

,AIC,BIC,FPE,HQIC
1,-19.56,-19.32,3.191e-09,-19.47
2,-20.83,-19.11,9.110e-10,-20.20
3,-21.25,-18.06,6.611e-10,-20.08
4,-28.41,-23.74,6.786e-13,-26.69
5,-30.61,-24.47*,1.375e-13*,-28.35
6,-31.56*,-23.94,1.886e-13,-28.75*


In [7]:
positions = []
for i in y.columns:
    for m in range(1,len(y[i])+1):
        if y[i][m].__dict__['data'].find('*') == 10:
            positions.append(m)
mode_lag = int(stats.mode(positions)[0])


In [8]:
model_fitted = model.fit(mode_lag)

In [12]:
sumary2 = model_fitted.summary()

In [13]:
sumary2

  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Tue, 16, Feb, 2021
Time:                     19:33:20
--------------------------------------------------------------------
No. of Equations:         6.00000    BIC:                   -23.9448
Nobs:                     43.0000    HQIC:                  -28.7537
Log likelihood:           498.519    FPE:                1.88581e-13
AIC:                     -31.5630    Det(Omega_mle):     7.25968e-15
--------------------------------------------------------------------
Results for equation R0R4_3M
                              coefficient       std. error           t-stat            prob
-------------------------------------------------------------------------------------------
const                            0.000005         0.000018            0.300           0.764
L1.R0R4_3M                      -0.514554         0.380951           -1.351           0.177
L1.UK, GDP (q/

In [9]:
parameters = model_fitted.params

In [10]:
parameters = parameters['R0R4_3M']

In [11]:
# Check for Serial Correlation of Residuals (Errors) using Durbin Watson Statistic
from statsmodels.stats.stattools import durbin_watson
out = durbin_watson(model_fitted.resid)
for col, val in zip(actuals.columns, out):
    print(col, ':', round(val, 2))

R0R4_3M : 1.74
UK, GDP (q/q) : 2.16
UK, GDP (y/y) : 2.16
UK, unemployment rate : 2.56
UK, inflation (HICP) : 2.3
UK, home prices (y/y) : 1.46


In [201]:
#Serial correlation of the errors seem to be fine. 

In [202]:
parameters

const                       0.000005
L1.R0R4_3M                 -0.514554
L1.UK, GDP (q/q)            0.003005
L1.UK, GDP (y/y)           -0.003210
L1.UK, unemployment rate    0.000035
L1.UK, inflation (HICP)     0.000059
L1.UK, home prices (y/y)    0.000014
L2.R0R4_3M                 -0.487649
L2.UK, GDP (q/q)            0.003344
L2.UK, GDP (y/y)           -0.000495
L2.UK, unemployment rate   -0.000028
L2.UK, inflation (HICP)     0.000014
L2.UK, home prices (y/y)   -0.000009
L3.R0R4_3M                  0.007500
L3.UK, GDP (q/q)            0.003390
L3.UK, GDP (y/y)           -0.000013
L3.UK, unemployment rate    0.000312
L3.UK, inflation (HICP)     0.000047
L3.UK, home prices (y/y)    0.000002
L4.R0R4_3M                 -0.145030
L4.UK, GDP (q/q)            0.003511
L4.UK, GDP (y/y)           -0.000038
L4.UK, unemployment rate   -0.000074
L4.UK, inflation (HICP)     0.000031
L4.UK, home prices (y/y)    0.000024
L5.R0R4_3M                  0.188682
L5.UK, GDP (q/q)            0.000491
L

In [203]:
parameters.to_csv ('parameters.csv', index = True, header=True) 